<a href="https://colab.research.google.com/github/mohamedsayed18/Pose_estimation/blob/master/TFlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import cv2 
from matplotlib import pyplot as plt

In [0]:
#load my drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Load the image
img = cv2.imread("/content/drive/My Drive/innopolis/1st year courses/2nd term/CV/GP/data/im0002.jpg")
new_im = cv2.resize(img, (257,257))

In [0]:
# Load TFLite model and allocate tensors.
path = "/content/drive/My Drive/innopolis/1st year courses/2nd term/CV/GP/posenet.tflite"
interpreter = tf.lite.Interpreter(model_path=path)
interpreter.allocate_tensors()

In [0]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [0]:
# Test model on random input data.
input_shape = input_details[0]['shape']   # get the shape
input_index = interpreter.get_input_details()[0]['index']   # get the index
new_im = new_im.reshape((input_shape))  #reshape
x = np.asarray(new_im).astype(np.float32)   # convert to float32


interpreter.set_tensor(input_index, x)
interpreter.invoke()

In [95]:
# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
output_offset = interpreter.get_tensor(output_details[1]['index'])
print(output_offset.shape)


(1, 9, 9, 34)


In [0]:
# sigmoid function
def sigmoid(a):
  return 1/(1 + np.exp(-a))
# Reference
#https://www.geeksforgeeks.org/implement-sigmoid-function-using-numpy/

In [0]:
PARTS = {
    0: 'NOSE',
    1: 'LEFT_EYE',
    2: 'RIGHT_EYE',
    3: 'LEFT_EAR',
    4: 'RIGHT_EAR',
    5: 'LEFT_SHOULDER',
    6: 'RIGHT_SHOULDER',
    7: 'LEFT_ELBOW',
    8: 'RIGHT_ELBOW',
    9: 'LEFT_WRIST',
    10: 'RIGHT_WRIST',
    11: 'LEFT_HIP',
    12: 'RIGHT_HIP',
    13: 'LEFT_KNEE',
    14: 'RIGHT_KNEE',
    15: 'LEFT_ANKLE',
    16: 'RIGHT_ANKLE'
}

In [0]:
class KeyPoint():
    def __init__(self, index, pos, v):
        x, y = pos
        self.x = x
        self.y = y
        self.index = index
        self.body_part = PARTS.get(index)
        self.confidence = v

    def point(self):
        return int(self.y), int(self.x)

    def to_string(self):
        return 'part: {} location: {} confidence: {}'.format(
            self.body_part, (self.x, self.y), self.confidence)

In [0]:
def get_keypoints(heatmaps, offsets, output_stride=32):
        scores = sigmoid(heatmaps)
        num_keypoints = scores.shape[2]
        heatmap_positions = []
        offset_vectors = []
        confidences = []
        for ki in range(0, num_keypoints ):
            x,y = np.unravel_index(np.argmax(scores[:,:,ki]), scores[:,:,ki].shape)
            confidences.append(scores[x,y,ki])
            offset_vector = (offsets[y,x,ki], offsets[y,x,num_keypoints+ki])
            heatmap_positions.append((x,y))
            offset_vectors.append(offset_vector)
        image_positions = np.add(np.array(heatmap_positions) * output_stride, offset_vectors)
        keypoints = [KeyPoint(i, pos, confidences[i]) for i, pos in enumerate(image_positions)]
        return keypoints

In [105]:
#interpreting the output heatmap
# taking sigmoid of heatmap
scores = sigmoid(output_data[0])
offsets = output_offset[0]  
num_keypoints = scores.shape[2]

heatmap_positions = []
offset_vectors = []
confidences = []

for k in range(num_keypoints):
  x, y = np.unravel_index(np.argmax(scores[:,:,k]), scores[:,:,k].shape)
  confidences.append(scores[x,y,k])
  offset_vector = (offsets[y,x,k], offsets[y,x,num_keypoints+k])

  heatmap_positions.append((x,y))
  offset_vectors.append(offset_vector)

image_positions = np.add(np.array(heatmap_positions) * 32, offset_vectors)

keypoint = [KeyPoint(i, pos, confidences[i])for i, pos in enumerate(image_positions)]

#print(len(keypoint))
for i in range(17):
  print(keypoint[i].to_string())
#refrence
#https://stackoverflow.com/questions/60032705/how-to-parse-the-heatmap-output-for-the-pose-estimation-tflite-model
# how unraverl work https://stackoverflow.com/questions/48135736/what-is-an-intuitive-explanation-of-np-unravel-index

part: NOSE location: (63.61388683319092, 63.53660011291504) confidence: 0.0006441882578656077
part: LEFT_EYE location: (126.92070829868317, 254.4178764820099) confidence: 0.00033454925869591534
part: RIGHT_EYE location: (96.42712360620499, 67.15237879753113) confidence: 0.0005283950013108552
part: LEFT_EAR location: (95.18149185180664, 6.674205303192139) confidence: 0.00037403570604510605
part: RIGHT_EAR location: (123.7621078491211, 257.62202858924866) confidence: 0.0002461621770635247
part: LEFT_SHOULDER location: (121.60605001449585, -1.2256296873092651) confidence: 0.0003962194314226508
part: RIGHT_SHOULDER location: (182.8860902786255, 9.635981559753418) confidence: 0.0003691992606036365
part: LEFT_ELBOW location: (162.19057440757751, -1.6407703161239624) confidence: 0.0003833652299363166
part: RIGHT_ELBOW location: (160.14137637615204, 93.18889617919922) confidence: 0.0004271281068213284
part: LEFT_WRIST location: (130.3547921180725, -2.8289451599121094) confidence: 0.00053629965